In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StringType
from pyspark import SQLContext

conf= SparkConf().setMaster("local").setAppName("Mi programa")
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

dfspark = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("C:/Users/Mayo/Documents/base_datos_2008.csv")
dfspark.spark.sample(fraction = 0.001, withReplacement = False)
dfspark = dfspark.witColumn("ArrDelay", dfspark["ArrDelay"].cast("integer"))


df2 = dfspark.na.drop(subset = ["ArrDelay", "DepDelay", "Distance"])
df2 = df2.filter("ArrDelay is not Null")
df2 = df2.dropDuplicates()

In [ ]:
#Las funciones map son aquellas que procesan nuestros datos en particiones, aplicandoles filtros y ordenaciones
#Posteriormente las funciones reduce los procesan y los resumen

#Array parallelizado de la columna Origin
A = sc.parallelize(df2.select("Origin").rdd.collect())

In [ ]:
#Convertimos a persistente (acelera el proceso)
A.persist()

In [ ]:
mapfunction = A.map(lambda x: (x,1))
#Regresa el mismo objeto y el numero 1 

In [ ]:
mapfunction.collect()

In [ ]:
def fun(x):
    if x[0] in ["SEA", "ATL", "HOU"]:
        return((x,2))
    elif x[0] == "DEN":
        return((x,3))
    else:
        return((x,1))

In [ ]:
mapfunction2 = A.map(fun)

In [ ]:
mapfunction2.collect()

In [ ]:
#Crea una tabla de frecuencias para cada uno de los origenes
reducefunction = mapfunction.reduceByKey(lambda x,y: x+y)

In [ ]:
reducefunction2 = mapfunction2.reduceByKey(lambda x,y: x+y)

In [ ]:
reducefunction.collect()

In [ ]:
reducefunction2.collect()

In [ ]:
reducefunction.sortByKey().take(10)

In [ ]:
#La funcion lambda los ordena ascendentemente
reducefunction.sortBy(lambda x: x[1], ascending=False).take(10)